In [1]:
import numpy as np
import pandas as pd
import d6tflow as d6t
from tqdm import tqdm

Loading postgres module without psycopg2 nor pg8000 installed. Will crash at runtime if postgres functionality is used.
Loading S3 module without the python package boto3. Will crash at runtime if S3 functionality is used.


Welcome to d6tflow! For Q&A see https://github.com/d6t/d6tflow


In [3]:
from vaeps import predictions as pr
from vaeps import action_values as av

from soccer_mix.MixtureModels import dataprocessing as dp
from soccer_mix.MixtureModels import categoricalmodel as cm
from soccer_mix.MixtureModels import locationmodel as lm
from soccer_mix.MixtureModels import directionmodel as dm
from soccer_mix.MixtureModels import actions_clusters as ac
from soccer_mix.MixtureModels import sequences as sms

from ZoneSequences import actions_zones as az
from ZoneSequences import zone_sequences as zs

from mining import zone
from mining import soccermix as sm

C:\Users\Galo\Anaconda3\envs\HP\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [5]:
loc_models = lm.SelectLocationSoccerMix(competition='England').output().load()

In [11]:
loc_x = []
loc_y = []
for mod in loc_models:
    if mod.name != 'receival':
        for submod in mod.submodels:
            loc_x.append(submod.mean[0])
            loc_y.append(submod.mean[1])

In [9]:
loc_models[0].submodels[0].mean[0]

96.72496921168158

In [4]:
loc_weights = lm.LocationWeightsSoccerMix(competition='England').output().load()

In [10]:
receival_cols = [c for c in list(loc_weights.columns) if 'receival' in c]
loc_weights = loc_weights.drop(receival_cols, axis=1)
loc_weights = loc_weights[loc_weights.drop('action_id', axis=1).sum(axis=1) != 0]
loc_cols = list(loc_weights.drop('action_id', axis=1).columns)

In [13]:
loc_clusters = pd.DataFrame(data={
    'id': [i for i in range(1, len(loc_cols) + 1)],
    'name': loc_cols,
    'loc_x': loc_x,
    'loc_y': loc_y
})

In [15]:
loc_clusters.to_csv(path_or_buf=d6t.settings.dir + r'\ClusterAttributesSoccerMix\loc_clusters.csv')

In [14]:
dir_models = dm.SelectDirectionExperimentsSoccerMix(competition='England').output().load()

In [16]:
dir_weights = dm.DirectionWeightsSoccerMix(competition='England').output().load()

In [17]:
receival_cols = [c for c in list(dir_weights.columns) if 'receival' in c]
dir_weights = dir_weights.drop(receival_cols, axis=1)
dir_weights = dir_weights[dir_weights.drop('action_id', axis=1).sum(axis=1) != 0]
dir_cols = list(dir_weights.drop('action_id', axis=1).columns)

In [19]:
loc_sups = [mod.split('_')[0] + '_' + mod.split('_')[1] for mod in dir_cols]

In [28]:
loc_sups = []
for mod in dir_cols:
    name_split = mod.split('_')
    if name_split[0] != 'take':
        loc_sups.append(name_split[0] + '_' + name_split[1])
    else:
        loc_sups.append(name_split[0] + '_' + name_split[1] + '_' + name_split[2])

In [29]:
dir_clusters = pd.DataFrame(data={
    'id': [i for i in range(1, len(dir_cols) + 1)],
    'name': dir_cols,
    'loc_name': loc_sups,
})

In [30]:
dir_clusters = dir_clusters.merge(right=loc_clusters.drop('id', axis=1).rename(columns={'name': 'loc_name'}), on='loc_name')

In [31]:
dir_clusters.to_csv(path_or_buf=d6t.settings.dir + r'\ClusterAttributesSoccerMix\dir_clusters.csv')

In [33]:
grid_x = [
    10,
    10,
    10,
    28.125,
    28.125,
    28.125,
    28.125,
    28.125,
    44.375,
    44.375,
    44.375,
    44.375,
    44.375,
    60.625,
    60.625,
    60.625,
    60.625,
    60.625,
    76.875,
    76.875,
    76.875,
    76.875,
    76.875,
    95,
    95,
    95,
    95,
    95
]

In [34]:
grid_y = [
    61,
    34,
    7,
    61,
    48.5,
    34,
    19.5,
    7,
    61,
    48.5,
    34,
    19.5,
    7,
    61,
    48.5,
    34,
    19.5,
    7,
    61,
    48.5,
    34,
    19.5,
    7,
    61,
    48.5,
    34,
    19.5,
    7
]

In [35]:
grid_cells = pd.DataFrame(data={
    'id': [i for i in range(1, len(grid_x) + 1)],
    'grid_x': grid_x,
    'grid_y': grid_y
})

In [36]:
grid_cells.to_csv(path_or_buf=d6t.settings.dir + r'\ClusterAttributesSoccerMix\grid_cells.csv')